# Client Version History

## Investigate the Observations Database created by the scheduling-bazaar scripts.

In [1]:
%load_ext sql

In [2]:
# %sql sqlite:////mnt/home/c4/satnogs/data/all/observations.db
%sql sqlite:////home/kerel/observations_2023-04-07/observations.db

In [27]:
%%sql
PRAGMA table_info([observations]);

   sqlite:////home/kerel/observations_2023-04-07/observations.db
 * sqlite:////mnt/home/c4/satnogs/data/all/observations.db
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,0,None,1
1,start,TEXT,0,None,0
2,end,TEXT,0,None,0
3,ground_station,INTEGER,0,None,0
4,transmitter,TEXT,0,None,0
5,norad_cat_id,INTEGER,0,None,0
6,payload,TEXT,0,None,0
7,waterfall,TEXT,0,None,0
8,demoddata,TEXT,0,None,0
9,station_name,TEXT,0,None,0


In [22]:
%%sql
.indexes

 * sqlite:////home/kerel/observations_2023-04-07/observations.db
   sqlite:////mnt/home/c4/satnogs/data/all/observations.db
(sqlite3.OperationalError) near ".": syntax error
[SQL: .indexes]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [4]:
%%sql
SELECT
    ground_station,
    station_name,
    COUNT(id) as count
FROM observations
WHERE
    start >= '2023-04-04' AND
    start < '2023-04-05'
GROUP BY ground_station
ORDER BY count DESC
LIMIT 1000;

In [5]:
print(f'There have been {len(_)} active stations on 2023-04-04.')

In [6]:
%%sql
SELECT
    status,
    COUNT(id) as count
FROM observations
WHERE
    start >= '2023-04-04' AND
    start < '2023-04-05'
GROUP BY status
LIMIT 1000;

In [7]:
%%sql
SELECT
    client_version,
    COUNT(1) as count
FROM observations
WHERE
    start >= '2023-04-04' AND
    start < '2023-04-05' AND
    status == 'good'
GROUP BY client_version;

In [8]:
versions = {}
versions['2023-04-04'] = _

In [9]:
%%sql
SELECT
    client_version,
    ground_station,
    station_name
FROM observations
WHERE
    start >= '2023-01-04' AND
    start < '2023-01-05' AND
    status == 'good'
GROUP BY ground_station;

## Sample client version of each active station for one day of the month

In [9]:
import sqlite3
from collections import defaultdict
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import pytz

In [10]:
# connection = sqlite3.connect("/mnt/home/c4/satnogs/data/all/observations.db")
connection = sqlite3.connect("/home/kerel/observations_2023-04-07/observations.db")
cursor = connection.cursor()

In [11]:
statement = """
SELECT
    date,
    client_version,
    count(*) as active_stations
FROM (
    WITH RECURSIVE dates(date) AS (
        VALUES('2014-01-04')
        UNION ALL
        SELECT
            date(date, '+1 month')
        FROM
            dates
        WHERE
            date < '2023-01-06'
    )
    SELECT
        dates.date,
        client_version,
        ground_station
    FROM times_index
    INNER JOIN dates
    -- ON start BETWEEN dates.date AND date(dates.date, '+1 day')
    ON start >= strftime("%s", dates.date) AND start < strftime("%s", date(dates.date, '+1 day'))
    WHERE status == 'good'
    GROUP BY dates.date, client_version, ground_station
)
GROUP BY date, client_version
ORDER BY date, client_version ASC;
"""
rows = cursor.execute(statement).fetchall()

In [14]:
df_adoption = pd.DataFrame.from_records(rows, columns=['date', 'tag', 'adoption'])
df_adoption.to_parquet('df_adoption.gzip')

In [15]:
df_adoption

,date,tag,adoption
0,2016-03-04,,1
1,2016-06-04,,1
2,2017-02-04,,1
3,2017-03-04,,1
4,2017-04-04,,1
...,...,...,...
950,2023-02-04,1.8.1,56
951,2023-02-04,1.8.1+27.gffccea9,4
952,2023-02-04,1.8.1+28.g5f23fe0,4
953,2023-02-04,1.8.1+30.g1dec8ce,1


In [6]:
version_history = defaultdict(dict)

for date, tag, count in rows:
    if tag in version_history[date]:
        print('Error')
    version_history[date][tag] = count

df = pd.DataFrame(version_history)
df

,2016-03-04,2016-06-04,2017-02-04,2017-03-04,2017-04-04,2017-05-04,2017-06-04,2017-07-04,2017-08-04,2017-09-04,...,2022-07-04,2022-08-04,2022-09-04,2022-10-04,2022-11-04,2022-12-04,2023-01-04,2023-02-04,2023-03-04,2023-04-04
,3.0,1.0,11.0,9.0,39.0,14.0,72.0,73.0,126.0,35.0,...,11.0,8.0,7.0,13.0,10.0,14.0,27.0,8.0,11.0,11.0
0.6.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0-pre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.6.2+0.gbdfaf21.dirty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.7+21.g291ea16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,7.0,3.0
1.8.1+28.g5f23fe0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,215.0,73.0,94.0,1.0
1.8.1+30.g1dec8ce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,NaN,NaN
1.8.1+34.gf30cc9a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,71.0


In [7]:
reference_date = '2023-04-04'
threshold = 0.5

df_sorted = df.sort_values(by=reference_date, ascending=False)
df_short = df_sorted[df_sorted[reference_date].rank(pct=True) > threshold]

# Hack....
df_others_row = pd.DataFrame(df_sorted[df_sorted.rank(pct=True) <= threshold].sum().transpose(), columns=['others']).transpose()

df_short = pd.concat([df_short, df_others_row])
df_short

,2016-03-04,2016-06-04,2017-02-04,2017-03-04,2017-04-04,2017-05-04,2017-06-04,2017-07-04,2017-08-04,2017-09-04,...,2022-07-04,2022-08-04,2022-09-04,2022-10-04,2022-11-04,2022-12-04,2023-01-04,2023-02-04,2023-03-04,2023-04-04
1.8.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,451.0,682.0,707.0,774.0,676.0,777.0,775.0
1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1229.0,1440.0,1459.0,635.0,615.0,907.0,845.0,540.0,505.0,489.0
1.8.1+27.gffccea9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,160.0,145.0,140.0,159.0,190.0
0+unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,61.0,63.0,92.0,101.0
1.8.1+34.gf30cc9a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,71.0
0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,3.0,NaN,NaN,NaN,52.0,28.0,51.0,33.0,69.0
1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,46.0,38.0,97.0,74.0,94.0,99.0,87.0,70.0,105.0,67.0
1.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,40.0,9.0,15.0,13.0,36.0,53.0,49.0,27.0
1.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,153.0,246.0,129.0,78.0,44.0,44.0,42.0,26.0,20.0,19.0
1.5.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.0,6.0,12.0,13.0,5.0,6.0,11.0,16.0,19.0,18.0


## Load Release dates from satnogs-client git history

Extract the release dates from git history with the following command:
```bash
TZ=UTC0 git for-each-ref --sort=creatordate --format "%(refname), %(creatordate:iso)" refs/tags > /mnt/home/c4/satnogs/python/scheduling-bazaar/satnogs-client-releases.csv
```

In [6]:
release_dates = []
with open('../satnogs-client-releases.csv') as fdata:
    for line in fdata:
        ref, date_str = line.split(',')
        version = ref[10:]

        if '+' in date_str or '-' in date_str:
            # Handle timezones
            sign = '+' if '+' in date_str else '-'
            date_lcl, tz = date_str.strip().split(f' {sign}')
            date_str_iso = f'{date_lcl}{sign}{tz[:2]}:{tz[2:]}'
        else:
            date_str_iso = date_str.strip()

        date = datetime.fromisoformat(date_str_iso).astimezone(pytz.utc).replace(tzinfo=None)
        release_dates.append([version, date])

        # print(f'{version:9s} {date}')

df_versions = pd.DataFrame.from_records(release_dates, columns=['tag', 'release_date'], index='tag')
df_versions

,release_date
tag,
0.1,2015-01-19 22:20:39
v0.1,2015-01-19 22:20:39
0.2,2015-03-11 20:13:30
v0.2,2015-03-11 20:13:30
0.2pypi,2015-03-14 18:23:30
0.2.1,2015-05-03 17:16:10
v0.2.1,2015-05-03 17:16:10
0.2.2,2015-05-12 18:09:12
v0.2.2,2015-05-12 18:09:12


## Store df_short and df_versions

In [10]:
df_versions.to_parquet('df_versions.gzip')

In [8]:
df_short.to_parquet('df_short3.gzip')

In [39]:
%%sql
DROP TABLE sample_dates;

   sqlite:////home/kerel/observations_2023-04-07/observations.db
 * sqlite:////mnt/home/c4/satnogs/data/all/observations.db
(sqlite3.OperationalError) no such table: sample_dates
[SQL: DROP TABLE sample_dates;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [48]:
%%sql

SELECT 
    min_date as date,
    (SELECT COUNT(start) FROM observations
     WHERE start BETWEEN min_date AND max_date) as count
FROM
(WITH RECURSIVE dates(date) AS (
  VALUES('2023-01-04')
  UNION ALL
  SELECT date(date, '+1 month')
  FROM dates
  WHERE date < '2023-12-31'
)
SELECT date as min_date, date(date, '+1 day') as max_date FROM dates)

   sqlite:////home/kerel/observations_2023-04-07/observations.db
 * sqlite:////mnt/home/c4/satnogs/data/all/observations.db
Done.


date,count
2023-01-04,5367
2023-02-04,3820
2023-03-04,4215
2023-04-04,4642
2023-05-04,0
2023-06-04,0
2023-07-04,0
2023-08-04,0
2023-09-04,0
2023-10-04,0


In [87]:
%%sql

WITH dates(min_date, max_date) AS(
    SELECT 
        min_date,
        max_date
    FROM (
        WITH RECURSIVE dates(date) AS (
            VALUES('2023-01-04')
            UNION ALL
            SELECT
                date(date, '+1 month')
            FROM
                dates
            WHERE
                date < '2023-12-31'
        )
        SELECT
            date as min_date,
            date(date, '+1 day') as max_date
        FROM dates
    )
)
SELECT
    dates.min_date
FROM
dates;

   sqlite:////home/kerel/observations_2023-04-07/observations.db
 * sqlite:////mnt/home/c4/satnogs/data/all/observations.db
Done.


min_date
2023-01-04
2023-02-04
2023-03-04
2023-04-04
2023-05-04
2023-06-04
2023-07-04
2023-08-04
2023-09-04
2023-10-04


In [110]:
%%sql

SELECT
    date,
    client_version,
    COUNT(ground_station) as adoption
FROM
    (SELECT
        '2023-01-04' as date,
        client_version,
        ground_station
    FROM observations
    WHERE
        start >= date AND
        start < date(date, '+1 day') AND
        status == 'good'
    GROUP BY ground_station)
GROUP BY client_version

   sqlite:////home/kerel/observations_2023-04-07/observations.db
 * sqlite:////mnt/home/c4/satnogs/data/all/observations.db
Done.


date,client_version,adoption
2023-01-04,,6
2023-01-04,0+unknown,1
2023-01-04,0.3+336.gb5a8aa4,1
2023-01-04,0.9,4
2023-01-04,0.9.1,2
2023-01-04,1.0,5
2023-01-04,1.1.2,1
2023-01-04,1.2,1
2023-01-04,1.3.1,3
2023-01-04,1.3.4,3


In [3]:
%%sql

DROP TABLE IF EXISTS SampleDates;
CREATE TEMP TABLE SampleDates(date);

WITH RECURSIVE dates(date) AS (
    VALUES('2023-01-04')
    UNION ALL
    SELECT
        date(date, '+1 month')
    FROM
        dates
    WHERE
        date < '2023-12-31'
)
INSERT INTO SampleDates
SELECT
    date
FROM dates
WHERE true;

 * sqlite:////mnt/home/c4/satnogs/data/all/observations.db
Done.
Done.
Done.


[]

In [4]:
%%sql

SELECT
    date
FROM
    SampleDates

 * sqlite:////mnt/home/c4/satnogs/data/all/observations.db
Done.


date
2023-01-04
2023-02-04
2023-03-04
2023-04-04
2023-05-04
2023-06-04
2023-07-04
2023-08-04
2023-09-04
2023-10-04


In [49]:
%%sql

SELECT
    SampleDates.date,
    observations.client_version,
    COUNT(*) as adoption
FROM observations
INNER JOIN SampleDates
ON observations.start BETWEEN SampleDates.date AND date(SampleDates.date, '+1 day')
WHERE observations.status == 'good'
GROUP BY SampleDates.date, observations.client_version;

 * sqlite:////mnt/home/c4/satnogs/data/all/observations.db
Done.


date,client_version,adoption
2023-01-04,,26
2023-01-04,0+unknown,61
2023-01-04,0.3+336.gb5a8aa4,2
2023-01-04,0.9,27
2023-01-04,0.9.1,11
2023-01-04,1.0,6
2023-01-04,1.1.2,3
2023-01-04,1.2,3
2023-01-04,1.3.1,10
2023-01-04,1.3.4,15


In [47]:
%%sql
SELECT
    observations.start,
    observations.client_version
FROM observations
    WHERE
        observations.start BETWEEN '2023-01-04' AND date('2023-01-04', '+1 day') AND
        status == 'good'
ORDER BY observations.start DESC
LIMIT 10;

 * sqlite:////mnt/home/c4/satnogs/data/all/observations.db
Done.


start,client_version
2023-01-04T23:52:32Z,1.8.1
2023-01-04T23:52:31Z,1.8.1
2023-01-04T23:51:59Z,1.6
2023-01-04T23:51:45Z,1.8.1+27.gffccea9
2023-01-04T23:51:45Z,1.8.1+27.gffccea9
2023-01-04T23:51:25Z,1.6
2023-01-04T23:50:02Z,1.8.1
2023-01-04T23:49:17Z,0.9
2023-01-04T23:48:53Z,1.5.1
2023-01-04T23:47:48Z,1.8.1


In [60]:
%%sql

SELECT
    date,
    client_version,
    count(*) as active_stations
FROM (
    WITH RECURSIVE dates(date) AS (
        VALUES('2014-01-04')
        UNION ALL
        SELECT
            date(date, '+1 month')
        FROM
            dates
        WHERE
            date < '2023-01-06'
    )
    SELECT
        dates.date,
        client_version,
        ground_station
    FROM times_index
    INNER JOIN dates
    -- ON start BETWEEN dates.date AND date(dates.date, '+1 day')
    ON start >= strftime("%s", dates.date) AND start < strftime("%s", date(dates.date, '+1 day'))
    WHERE status == 'good'
    GROUP BY dates.date, client_version, ground_station
)
GROUP BY date, client_version
ORDER BY date, client_version ASC;

 * sqlite:////mnt/home/c4/satnogs/data/all/observations.db
Done.


date,client_version,active_stations
2020-06-04,,62
2020-06-04,0.3+271.g0bb7196,1
2020-06-04,0.7,4
2020-06-04,0.8,8
2020-06-04,0.9,39
2020-06-04,0.9.1,9
2020-06-04,1.0,25
2020-06-04,1.0.dev+177.g0aa8d19,1
2020-06-04,1.1,2
2020-06-04,1.1.2,5


In [3]:
%%sql

DROP TABLE IF EXISTS times_index;

-- Special index for speeding up time range queries.
CREATE VIRTUAL TABLE IF NOT EXISTS times_index
USING rtree(
        id,
        start,
        end,
        +ground_station INT,
        +client_version INT,
        +status INT
        );

INSERT OR REPLACE INTO times_index
SELECT
    id,
    strftime("%s", start),
    strftime("%s", end),
    ground_station,
    client_version,
    status
FROM observations
WHERE strftime("%s", start) <= strftime("%s", end);

 * sqlite:////home/kerel/observations_2023-04-07/observations.db
Done.
Done.
7270712 rows affected.


[]

In [57]:
%%sql

SELECT start, client_version, ground_station FROM times_index LIMIT 10;

 * sqlite:////mnt/home/c4/satnogs/data/all/observations.db
Done.


start,client_version,ground_station
1591228672.0,0.9,901
1591228800.0,1.3.1,1416
1591228800.0,1.2,1093
1591228800.0,1.2,1145
1591228800.0,1.3.1,1376
1591228800.0,1.1.2,1382
1591228800.0,0.9.1,329
1591228928.0,1.3.1,812
1591228928.0,1.3.1,91
1591228928.0,1.3.1,1337
